## Correcting local biases in sampling

In [1]:
import scipy
import numpy as np
import pandas as pd

from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import scale

from scipy.stats.stats import pearsonr 

from scipy.stats import invgamma 
from scipy.stats import beta
import matplotlib.pyplot as plt

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)

In our first post we examined how to use frequency vectors to generate haplotypes and populations. We then proceeded to generate a universe of frequency vectors, whose distance in feature space allowed us to chose the relative differentiation of the populations we would simulate.

What i didn't touch on in that post was the importance of sampling in principal component analysis. In the last section, i chose vectors close to one another, together with vectors far distant, in order to produce differentiated populations. If you tweeked the population sizes, you might have noticed that if some of the close together populations largely outweighed the rest, the distances to the more differentiated clusters would be reduced.

- see [McVean 2009](http://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1000686) for a study case.

Let's do that now. As in the first post, we will start by generating a space of vectors. Chose at least two to be far appart, and the rest to be closer together. Give at least one of the latter an innordinate size difference to the rest.

In [2]:
# Simulate frequency vectors. 
# We must first define the number of populations, the length of the haplotypes desired, and their respective population sizes
L= 200

import itertools as it
n= 10

# Vary a (beta distribution parameter).
a_range= np.linspace(1,2,11)
a_set= [i for i in a_range for _ in range(n)]

# vary b.
b_range= np.linspace(0.1,.4,11)
b_set= [i for i in b_range for _ in range(n)]

## length of haplotypes to extract.
L_set= [L] * n * 11


background= np.array([a_set,b_set,L_set]).T

vector_lib= []
for k in range(background.shape[0]):
    
    probs= beta.rvs(background[k,0], background[k,1], size=int(background[k,2]))
    probs[(probs > 1)]= 1
    
    
    vector_lib.append(probs)

vector_lib= np.array(vector_lib)

In [3]:
vector_lib.shape

(110, 200)

In [4]:
## PCA on vectors simulated
n_comp = 3

pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(vector_lib)
features = pca.transform(vector_lib) * pca.explained_variance_ratio_

print("; ".join(['PC{0}: {1}'.format(x+1,round(pca.explained_variance_ratio_[x],3)) for x in range(n_comp)]))
print('features shape: {}'.format(features.shape))

PC1: 0.03; PC2: 0.028; PC3: 0.026
features shape: (110, 3)


In [5]:
## Plot vector PCA
fig_data= [go.Scatter3d(
        x = features[:,0],
        y = features[:,1],
        z = features[:,2],
        type='scatter3d',
        mode= "markers",
        text= ['a: {}; b: {}, L: {}; index = {}'.format(background[k,0],background[k,1],background[k,2], k) for k in range(background.shape[0])],
        marker= {
        'line': {'width': 0},
        'size': 4,
        'symbol': 'circle',
      "opacity": .8
      }
    )]


layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)


 **Fig. 1** PCA on frequency vectors generated from the beta distribution. Parameters a and b were made to vary between 
1-2 and .1-.4 respectively at steps of .1. For each combination of parameters 15 vectors were produced.


The point of this is to peruse frequency vector space, and chose populations. Because PCA distances equal correlation between observations, the closer the points the higher the Fst between the pops selected.


### I. Biased sampling of frequency vectors
chose populations and bias sizes below.

Haplotypes for each population will be generated using the frequency vectors as the probabilities of alleles.

In [6]:
### Select frequency vectors and draw haplotypes.
## Pops selected by Indicies.
Pops= [99,95,109,17,55]
N_pops= len(Pops)

## Population Sizes and labels
Sizes_bias= [130,80,300,35,50]
labels_bias= np.repeat(np.array([x for x in range(N_pops)]),Sizes_bias)

## Number of pops

data_ex= []

for k in range(N_pops):
    
    probs= vector_lib[Pops[k],:]
    
    m= Sizes_bias[k]
    Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]
    
    data_ex.extend(Haps)

data_ex= np.array(data_ex)
print(data_ex.shape)


(595, 200)


We will also calculate genetic distances between pairs of individuals based on the haplotypes. Calculation will be simple, distance based (n_diffs / hap_length).

In [7]:
### Calculate individual pairwise distances for biased sampling.
def pairwise_gen(x,y):
    miss= 0
    same= 0
    if len(x) != len(y):
        return 'vector lengths differ'
    else:
        for n in range(len(x)):
            if x[n] == y[n]:
                same += 1
        return 1 - same / (len(x) - miss)

bias_gen_diffs= pairwise_distances(data_ex,metric= pairwise_gen)
bias_gen_diffs= np.array(bias_gen_diffs)

iugen= np.triu_indices(bias_gen_diffs.shape[0],1)
bias_gen_diffs= bias_gen_diffs[iugen]

From the frequency vectors selected we can also calculate pairwise Fst's.

These will be used later to compare to pairswise centroid distances in feature space.

In [8]:
### Calculate pairwise Fst based on frequency vectors selected.

def return_fsts2(freq_array):
    pops= range(freq_array.shape[0])
    H= {pop: [1-(freq_array[pop,x]**2 + (1 - freq_array[pop,x])**2) for x in range(freq_array.shape[1])] for pop in range(freq_array.shape[0])}
    Store= []

    for comb in it.combinations(H.keys(),2):
        P= [sum([freq_array[x,i] for x in comb]) / len(comb) for i in range(freq_array.shape[1])]
        HT= [2 * P[x] * (1 - P[x]) for x in range(len(P))]
        per_locus_fst= [[(HT[x] - np.mean([H[p][x] for p in comb])) / HT[x],0][int(HT[x] == 0)] for x in range(len(P))]
        per_locus_fst= np.nan_to_num(per_locus_fst)
        Fst= np.mean(per_locus_fst)

        Store.append([comb,Fst])
    
    
    ### total fst:
    P= [sum([freq_array[x,i] for x in pops]) / len(pops) for i in range(freq_array.shape[1])]
    HT= [2 * P[x] * (1 - P[x]) for x in range(len(P))]
    FST= np.mean([(HT[x] - np.mean([H[p][x] for p in pops])) / HT[x] for x in range(len(P))])
    
    return pd.DataFrame(Store,columns= ['pops','fst'])

freqs_selected= vector_lib[Pops,:]
Pairwise= return_fsts2(freqs_selected)

fsts_compare = scale(Pairwise.fst)
Pairwise

,pops,fst
0,"(0, 1)",0.114017
1,"(0, 2)",0.106815
2,"(0, 3)",0.103648
3,"(0, 4)",0.102597
4,"(1, 2)",0.124005
5,"(1, 3)",0.138839
6,"(1, 4)",0.132895
7,"(2, 3)",0.126557
8,"(2, 4)",0.138045
9,"(3, 4)",0.112727


Finally we perform PCA on the haplotypes generated and keep the first 5 PCs. Notice that right now the multiplication by eigenvalues is commented out. 

To run this test with the multiplication by eigenvalues you can uncomment that (i'll write something simpler later on).

In [9]:
### PCA on haplotypes drawn.
n_comp = 5

pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data_ex)

bias_features= pca.transform(data_ex)# * pca.explained_variance_ratio_

var_comps= pca.explained_variance_ratio_
print("; ".join(['PC{0}: {1}'.format(x+1,round(var_comps[x],3)) for x in range(n_comp)]))
print(bias_features.shape)

PC1: 0.133; PC2: 0.064; PC3: 0.042; PC4: 0.027; PC5: 0.012
(595, 5)


Estimating population centroids in feature space and plot the PCA on biased-haplotype data set

In [10]:
## Calculate centroids
bias_centroids= [np.mean(bias_features[[y for y in range(bias_features.shape[0]) if labels_bias[y] == z],:],axis= 0) for z in range(N_pops)]
bias_centroids= np.array(bias_centroids)

## plot
fig_data= [go.Scatter(
        x = bias_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],0],
        y = bias_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],1],       
        type='scatter',
        mode= "markers",
        marker= {
        'line': {'width': 0},
        'size': 8,
        'symbol': 'circle',
      "opacity": .8
      },
      name= str(i)
    ) for i in range(N_pops)]


fig_data.append(
    go.Scatter(
        x= bias_centroids[:,0],
        y= bias_centroids[:,1],
        type= 'scatter',
        mode= 'markers',
        name= 'centres',
        marker= {
        'line': {'width': 1},
        'size': 10,
        'symbol': 'cross'
        }
    )
)

layout = go.Layout(
    title= 'Biased sampling; eigenvalues factored in',
    yaxis=dict(
        title='PC2: {}'.format(round(var_comps[1],3))),
    xaxis=dict(
    title= 'PC1: {}'.format(round(var_comps[0],3))),
)


fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 2** PCA on haplotypes sampled unevenly from the frequency vectors selected above. 

Calculate Pairwise centroid distances in feature space (remember we kept 5 components), and individual pairwise distances in this space also.

In [11]:
## centroid distances
iu1= np.triu_indices(N_pops,1)
bias_pair_dist= pairwise_distances(bias_centroids,metric= 'euclidean')
bias_pair_dist= bias_pair_dist[iu1]
bias_pair_dist= scale(bias_pair_dist)

## Individual distances:
bias_feat_dist= pairwise_distances(bias_features, metric= 'euclidean')
bias_feat_dist= bias_feat_dist[iugen]
bias_feat_dist= scale(bias_feat_dist)


### Pearson's r between individual pairwise genetic and feature space distances.
bias_feat_Pearson= pearsonr(bias_feat_dist,bias_gen_diffs)
print('Our first result: Pearson r between individual feature space and genetic distances: {}'.format(bias_feat_Pearson[0]))


Our first result: Pearson r between individual feature space and genetic distances: 0.8272954117376633


I chose the first two populations to play the outliers, the rest to be a close pack. To two of these i gave population sizes of 300 and 180, six and 3.6 times the size of the largest outlying population. 

The distortion can be seen in that our outlying populations don't appear as far as we would have expected them to given their vectors alone. They tend to appear in the center because of their reduced impact on variance components.

As remarked by McVean, this can be a problem when deriving conclusions from relative distances in feature space.


### II. MeanShift correction.

My approach here isn't very elegant. MeanShift allows us to identify clusters in feature space, i just resample those clusters equally, inverse transform their coordinates and perform the PCA anew. The actual data is transposed onto the resulting space.

In [12]:
def local_sampling_correct(data_now,n_comp):
    pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data_now)
    feats= pca.transform(data_now)
    
    N= 50
    bandwidth = estimate_bandwidth(feats, quantile=0.2)
    params = {'bandwidth': np.linspace(np.min(feats), np.max(feats),30)}
    grid = GridSearchCV(KernelDensity(algorithm = "ball_tree",breadth_first = False), params,verbose=0)
    
    ## perform MeanShift clustering.
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, cluster_all=False, min_bin_freq=5)
    ms.fit(feats)
    labels1 = ms.labels_
    label_select = {y:[x for x in range(len(labels1)) if labels1[x] == y] for y in sorted(list(set(labels1))) if y != -1}

    ## Extract the KDE of each cluster identified by MS.
    Proxy_data= []

    for lab in label_select.keys():
        if len(label_select[lab]) < 3:
            continue
            
        Quanted_set= feats[label_select[lab],:]
        grid.fit(Quanted_set)

        kde = grid.best_estimator_
        Extract= kde.sample(N)
        Return= pca.inverse_transform(Extract)
        
        #Return= data_now[np.random.choice(label_select[lab],N),:]
        Proxy_data.extend(Return)
    
    Proxy_data= np.array(Proxy_data)
    
    print([len(x) for x in label_select.values()])
    pca2 = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(Proxy_data)
    var_comp= pca2.explained_variance_ratio_
    
    New_features= pca2.transform(data_now) * var_comp
    return New_features, var_comp


New_features,var_comp= local_sampling_correct(data_ex,5)

[299, 130, 80, 50, 35]


Plotting our original samples onto our re-computed feature space:

In [14]:
corr_centroids= [np.mean(New_features[[y for y in range(New_features.shape[0]) if labels_bias[y] == z],:3],axis= 0) for z in range(N_pops)]
corr_centroids= np.array(corr_centroids)

fig_data= [go.Scatter(
        x = New_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],0],
        y = New_features[[x for x in range(sum(Sizes_bias)) if labels_bias[x] == i],1],
        type='scatter',
        mode= "markers",
        marker= {
        'line': {'width': 0},
        'size': 8,
        'symbol': 'circle',
      "opacity": .8
      },
      name= str(i)
    ) for i in range(N_pops)]

fig_data.append(
    go.Scatter(
        x= corr_centroids[:,0],
        y= corr_centroids[:,1],
        type= 'scatter',
        mode= 'markers',
        name= 'centres',
        marker= {
        'line': {'width': 1},
        'size': 10,
        'symbol': 'cross'
        }
    )
)


layout = go.Layout(
    title= 'Biased corrected, eigenvalues not factored in',
    yaxis=dict(
        title='PC2: {}'.format(round(var_comp[1],3))),
    xaxis=dict(
    title= 'PC1: {}'.format(round(var_comp[0],3))),
)

fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 3** MS corrected PCA of unvenly sampled haplotypes. MS clustering was first applied following an initial PCA of the biased data set. The KDE of each cluster identified this way was used to resample equally from that distribution. 50 observations were resampled from the distribution of each cluster identified by MS. Each observation from the resulting data set was inverse transformed and a new PCA was conducted on this data. Finally, the original haplotypes were projected onto the new feature space.

Very different from the original PCA on the biased data set.

We now calculate the distances between centroids in this feature space and the pairwise individual distances, compare the latter to genetic distances.

In [18]:
iu1= np.triu_indices(N_pops,1)
corrected_pair_dist= pairwise_distances(corr_centroids,metric= 'euclidean')
corrected_pair_dist= corrected_pair_dist[iu1]
corrected_pair_dist= scale(corrected_pair_dist)

## Individual distances:
corrected_feat_dist= pairwise_distances(New_features, metric= 'euclidean')
corrected_feat_dist= corrected_feat_dist[iugen]
corrected_feat_dist= scale(corrected_feat_dist)

corrected_gen_pearson= pearsonr(corrected_feat_dist,bias_gen_diffs)

print('Pearon r of individual genetic distances versus feature space distances following correction: {}'.format(round(corrected_gen_pearson[0],3)))


Pearon r of individual genetic distances versus feature space distances following correction: 0.803


### III. Even sampling.


We can compare this output with what we would have gotten from sampling equally across our selected vectors.

For this purpose we sample equally from the same frequency vectors as in the biased scenario and perform PCA on the resulting data set.

In [20]:
#### Selecting new, equal sample sizes but derive haplotypes from the same frequency vectors.

Sizes= [50,50,50,50,50]
labels= np.repeat(np.array([x for x in range(N_pops)]),Sizes)

data= []

for k in range(N_pops):
    
    probs= vector_lib[Pops[k],:]
    
    m= Sizes[k]
    Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]
    
    data.extend(Haps)

data= np.array(data)


#### clalculate pairwise genetic distances
iugen_unbiased= np.triu_indices(data.shape[0],1)

unbias_gen_diffs= pairwise_distances(data,metric= pairwise_gen)
unbias_gen_diffs= np.array(unbias_gen_diffs)

unbias_gen_diffs= unbias_gen_diffs[iugen_unbiased]

### perform PCA

n_comp = 3

pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data)

features= pca.transform(data)# * pca.explained_variance_ratio_

var_comps= pca.explained_variance_ratio_
print("; ".join(['PC{0}: {1}'.format(x+1,round(var_comps[x],3)) for x in range(n_comp)]))
print(features.shape)


#### Calculate centroids of labelled data in feature space.
unbias_centroids= [np.mean(features[[y for y in range(features.shape[0]) if labels[y] == z],:],axis= 0) for z in range(N_pops)]
unbias_centroids= np.array(unbias_centroids)


#### Plot projections + Centroids.

fig_data= [go.Scatter(
        x = features[[x for x in range(sum(Sizes)) if labels[x] == i],0],
        y = features[[x for x in range(sum(Sizes)) if labels[x] == i],1],
        type='scatter',
        mode= "markers",
        marker= {
        'line': {'width': 0},
        'size': 8,
        'symbol': 'circle',
      "opacity": .8
      },
      name= str(i)
    ) for i in range(N_pops)]


fig_data.append(
    go.Scatter(
        x= unbias_centroids[:,0],
        y= unbias_centroids[:,1],
        type= 'scatter',
        mode= 'markers',
        name= 'centres',
        marker= {
        'line': {'width': 1},
        'size': 10,
        'symbol': 'cross'
        }
    )
)

layout = go.Layout(
    title= 'Unbiased sampling; eigenvalues factored in',
    yaxis=dict(
        title='PC2: {}'.format(round(var_comps[1],3))),
    xaxis=dict(
    title= 'PC1: {}'.format(round(var_comps[0],3))),
)

fig = go.Figure(data=fig_data, layout=layout)
iplot(fig)

PC1: 0.103; PC2: 0.093; PC3: 0.075
(250, 3)


**Fig. 4** PCA on evenly sampled haplotypes from the same frequency vectors as above. 50 haplotypes generated from each vector.

Calculate pairwise distances. compare to genetic distances.

In [24]:
unbias_pair_dist= pairwise_distances(unbias_centroids,metric= 'euclidean')
unbias_pair_dist= unbias_pair_dist[iu1]
unbias_pair_dist= scale(unbias_pair_dist)

## Individual distances:
unbiased_feat_dist= pairwise_distances(features, metric= 'euclidean')

unbiased_feat_dist= unbiased_feat_dist[iugen_unbiased]
unbiased_feat_dist= scale(unbiased_feat_dist)

unbiased_gen_pearson= pearsonr(unbiased_feat_dist,unbias_gen_diffs)

print('Pearson r on individual genetic and feature space distances in the unbiased sampling scenario: {}'.format(round(corrected_gen_pearson[0])))


Pearson r on individual genetic and feature space distances in the unbiased sampling scenario: 1.0


In [23]:
t= np.array([
    unbias_pair_dist,
    bias_pair_dist,
    corrected_pair_dist
]).T

fig_data= [go.Scatter(
    x= t[:,0],
    y= t[:,i],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['bias','corrected'][i-1]
    ) for i in [1,2]
]

layout = go.Layout(
    title= 'MS correction distances',
    yaxis=dict(
        title='biased and corrected distances'),
    xaxis=dict(
        title='unbiased distances')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

**Fig. 5** Relation between pairwise centroid distances in biased scenario versus biased and corrected scenarios. Distances were scaled.

In [48]:

fig_fsts= [go.Scatter(
    x= fsts_compare,
    y= t[:,i],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['unbiased','biased','corrected'][i-1]
    ) for i in [0,1,2]
]

layout = go.Layout(
    title= 'PCA to genetic distances',
    yaxis=dict(
        title='centrois distances in feature space'),
    xaxis=dict(
        title='normalized Fst')
)

fig= go.Figure(data=fig_fsts, layout=layout)
iplot(fig)

**Fig. 6** Relationship between Fst and feature space distances in the three scenarios considered: unbiased, biased and MS corrected.


Let's now repeat this process sequentially, to get an idea of how much this method actually corrects distances between pops.

At each repetition we will choose a fixed number of frequency vectors from the _Vector Universe_ created at the top of this page. We then perform a biased and an unbiased sampling of each, and perform PCA on both. For each scenario we calculate the pairwise eucledian distances between the centroids of populations and normalize them. 

We then apply the MScorrection to the feature space of the biased scenario and recalculate pairwise centroid distances and normalize them.

This will allow us to compare the unbiased distances to biased and corrected distances. Hopefully, we will have reduced the distortion produced by the biases in sampling.

In [62]:
### Select frequency vectors and draw haplotypes.


N_pops= 5 # Number of pops

n_comp= 5 # components to keep following PCA

Iter= 20 # repeats

N_sims= 100 # number of haplotypes to generate from each pop in the unbiased scenario.


## Population Sizes and labels
bias_scheme= [130,43,200,40,60]
unbiased_sheme= np.repeat(N_sims,N_pops)

bias_labels= np.repeat(np.array([x for x in range(N_pops)]),bias_scheme)
unbias_labels= np.repeat(np.array([x for x in range(N_pops)]),unbiased_sheme)

### store distances between centroids
biased_pairwise= []
unbiased_pairwise= []
corrected_pairwise= []

### store fsts
fst_store= []

### store Pearson's r comparing gen_diffs and feature space diffs across scenarios
biased_pears= []
corrected_pears= []
unbiased_pears= []

### triangular matrices extract.
iu1= np.triu_indices(N_pops,1) # for centroid comparison

iu_unbias= np.triu_indices(sum(unbiased_sheme),1)
iu_bias= np.triu_indices(sum(bias_scheme),1)

### proceed.

for rep in range(Iter):
    Pops= np.random.choice(vector_lib.shape[0],N_pops,replace= False)
    print(Pops)
    ########## FST
    
    freqs_selected= vector_lib[Pops,:]
    Pairwise= return_fsts2(freqs_selected)

    fsts_compare = scale(Pairwise.fst)
    
    fst_store.extend(fsts_compare)
    #########################################################
    ########### PCA ####################################
    #########################################################
    ############# unbiased sample
    
    #### generate data and perform PCA.
    data= []

    for k in range(N_pops):

        probs= vector_lib[Pops[k],:]
        
        m= unbiased_sheme[k]
        Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]

        data.extend(Haps)
    
    data1= np.array(data)
    pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data1)
    feat_unbias= pca.transform(data1) * pca.explained_variance_ratio_
    
    ####### centroid comparison
    unbias_centroids= [np.mean(feat_unbias[[y for y in range(feat_unbias.shape[0]) if unbias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
    unbias_centroids= np.array(unbias_centroids)
    
    unbias_pair_dist= pairwise_distances(unbias_centroids,metric= 'euclidean')
    unbias_pair_dist= unbias_pair_dist[iu1]
    
    unbias_pair_dist= scale(unbias_pair_dist)
    unbiased_pairwise.extend(unbias_pair_dist)
    
    ######## ind distances
    ### genetic data
    unbias_gen_diffs= pairwise_distances(data1,metric= pairwise_gen)
    unbias_gen_diffs= np.array(unbias_gen_diffs)
    unbias_gen_diffs= unbias_gen_diffs[iu_unbias]
    
    ## feature space
    unbiased_feat_dist= pairwise_distances(feat_unbias, metric= 'euclidean')
    unbiased_feat_dist= unbiased_feat_dist[iu_unbias]
    unbiased_feat_dist= scale(unbiased_feat_dist)

    unbiased_gen_pearson= pearsonr(unbiased_feat_dist,unbias_gen_diffs)
    
    unbiased_pears.append(unbiased_gen_pearson[0])
    
    #################################################
    ############## biased sample
    
    #### generate data and perform PCA
    data= []

    for k in range(N_pops):

        probs= vector_lib[Pops[k],:]

        m= bias_scheme[k]
        Haps= [[np.random.choice([1,0],p= [1-probs[x],probs[x]]) for x in range(L)] for acc in range(m)]

        data.extend(Haps)

    data2= np.array(data)
    
    pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized').fit(data2)
    feat_bias= pca.transform(data2) * pca.explained_variance_ratio_
    
    #### Centroid distances
    bias_centroids= [np.mean(feat_bias[[y for y in range(feat_bias.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
    bias_centroids= np.array(bias_centroids)
    
    bias_pair_dist= pairwise_distances(bias_centroids,metric= 'euclidean')
    bias_pair_dist= bias_pair_dist[iu1]
    bias_pair_dist= scale(bias_pair_dist)
    biased_pairwise.extend(bias_pair_dist)

    ######## Ind distances
    ### genetic data
    bias_gen_diffs= pairwise_distances(data2,metric= pairwise_gen)
    bias_gen_diffs= np.array(bias_gen_diffs)
    bias_gen_diffs= bias_gen_diffs[iu_bias]
    
    ## feature space
    biased_feat_dist= pairwise_distances(feat_bias, metric= 'euclidean')
    biased_feat_dist= biased_feat_dist[iu_bias]
    biased_feat_dist= scale(biased_feat_dist)

    biased_gen_pearson= pearsonr(biased_feat_dist,bias_gen_diffs)
    
    biased_pears.append(biased_gen_pearson[0])
    
    ###############################################################"
    ################## bias correct
    ### perform MS correction on biased samples
    feat_correct,var_comp= local_sampling_correct(data2,n_comp)
    
    ### centroid Distances
    centroids= [np.mean(feat_correct[[y for y in range(feat_correct.shape[0]) if bias_labels[y] == z],:],axis= 0) for z in range(N_pops)]
    centroids= np.array(centroids)
    pair_dist= pairwise_distances(centroids,metric= 'euclidean')
    pair_dist= pair_dist[iu1]
    pair_dist= scale(pair_dist)
    corrected_pairwise.extend(pair_dist)
    
    ######## Ind distances
    
    ## feature space
    corrected_feat_dist= pairwise_distances(feat_correct, metric= 'euclidean')
    corrected_feat_dist= corrected_feat_dist[iu_bias]
    corrected_feat_dist= scale(corrected_feat_dist)

    corr_gen_pearson= pearsonr(corrected_feat_dist,bias_gen_diffs)
    
    corrected_pears.append(corr_gen_pearson[0])
    
    
    t= np.array([
    fsts_compare,
    unbias_pair_dist,
    bias_pair_dist,
    pair_dist
    ]).T
    print(t)


[98 63 26 11 83]
[200, 130, 60, 43, 40]
[0.79209087872645778, 0.75723829254475927, 0.79943257060283202]
[[ 0.22645626  0.11496024  0.08362822  0.18356251]
 [-0.32270554 -0.94580129  2.04751878 -1.37180649]
 [ 1.57745444  0.95852313  0.20044956  0.65222716]
 [-0.25601605  0.59591629 -0.0869373  -0.38402087]
 [-1.80604215 -1.90973306 -0.15727116 -1.46284892]
 [-0.18147821 -0.43289473 -1.73033732 -0.46709969]
 [ 1.34488747  1.35562803 -0.70215711  1.3855419 ]
 [-1.3310789  -1.09862634  0.11012152 -0.70227472]
 [ 0.11415275  0.60480325  1.16662855  0.63995324]
 [ 0.63436993  0.75722447 -0.93164375  1.52676589]]
[28 93 45 31 50]
[200, 130, 60, 43, 40]
[0.83159710298305678, 0.70399187818302078, 0.89145700799702154]
[[ 0.7365631   0.45849909 -0.20761796 -0.29112614]
 [-1.0282141  -0.90250238  1.87241045 -1.01005719]
 [-1.14637596 -0.79077178 -0.40745981 -0.62428951]
 [ 0.63986429  0.8478599   0.34755287  0.60457993]
 [-0.06324233 -0.26280203  0.57291428 -0.2999219 ]
 [ 1.67939663  1.59700641 

c:\users\jgarcia\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning:

invalid value encountered in double_scalars



[200, 130, 60, 43, 40]
[0.79935120308164931, 0.64812077706924798, 0.82637109715550805]
[[  1.96828410e+00   1.52734411e+00   5.90738035e-02   2.18674110e+00]
 [ -6.62841809e-01  -1.24806894e+00   1.83482401e+00  -1.94439603e+00]
 [ -2.00888049e+00  -8.94312287e-01  -3.02670150e-01  -6.46139745e-02]
 [ -1.45372440e-01   5.87442751e-02   7.98960742e-02  -1.56207836e-02]
 [  5.14863190e-01   9.95099487e-01   5.58023949e-01   5.19382369e-01]
 [ -4.33898319e-01  -1.61744383e+00  -1.52213171e+00  -9.65794860e-01]
 [  1.57288548e-03   4.07607122e-01  -1.15663340e+00  -2.15785694e-01]
 [  3.63149681e-01   7.03367461e-01   6.06564019e-01   3.10128756e-01]
 [  8.95563858e-01   7.53828709e-01   9.66903678e-01   2.81748316e-01]
 [ -4.92440658e-01  -6.86166108e-01  -1.12385027e+00  -9.17892015e-02]]
[24 44 20 93 31]
[200, 130, 60, 43, 40]
[0.83233036898371238, 0.69151629201915699, 0.88652131113670418]
[[-2.43584336 -2.18969507 -0.66962427 -2.36439323]
 [ 0.62745425  0.57372731  2.13121275  0.754498

In [63]:
t= np.array([
    unbiased_pairwise,
    biased_pairwise,
    corrected_pairwise
]).T


In [64]:
pearsons= [pearsonr(fst_store,t[:,x])[0] for x in range(t.shape[1])]

fig_data= [go.Scatter(
    x= fst_store,
    y= t[:,i],
    mode= 'markers',
    marker= dict(
        color= i,
        opacity= .6
    ),
    name= ['unbiased','biased','corrected'][i] + ' r: {}'.format(round(pearsons[i],3))
    ) for i in [0,1,2]
]

layout = go.Layout(
    title= 'Feature space distances against fst across sampling scenarios; variance included',
    yaxis=dict(
        title='biased distances'),
    xaxis=dict(
        title='Fst')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

In [109]:
fig_data= [go.Scatter(
x= t[:,0],
y= t[:,2],
mode= 'markers'
) 
]


layout = go.Layout(
    title= 'MS correction test. Pearson r= {}'.format(round(pearsons[0],3)),
    yaxis=dict(
        title='corrected distances'),
    xaxis=dict(
        title='unbiased distances')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

In [119]:
P_stare= np.vstack([P_store_VARexc,P_store_VARinc])
P_stare.shape

Labels= np.repeat(['VARexc','VARinc'],Iter)

In [120]:
### Compare p-values of genetic to feature space comparisons across biased, unbiased and corrected schemes.
#####

P_store_VARexc= np.array([
    biased_pears,
    corrected_pears,
    unbiased_pears,
]
).T

box_names= ['biased','corrected','unbiased']

box_data= [go.Box(
    y= P_stare[:,i],
    x= Labels,
    name= box_names[i]
) for i in range(P_store.shape[1])]

layout= go.Layout(
    title= 'Genetic to Feature space Pearson p-values across settings',
    boxmode= 'group'
)

fig= go.Figure(data=box_data,layout= layout)
iplot(fig)

In [76]:
P_store

array([[ 0.9066842 ,  0.90642306,  0.84978766],
       [ 0.91646873,  0.91651775,  0.89216522],
       [ 0.82422035,  0.82427635,  0.81342105],
       [ 0.85966271,  0.85940631,  0.84218832],
       [ 0.87593998,  0.87585345,  0.87466112],
       [ 0.83718243,  0.83751185,  0.80077815],
       [ 0.87428352,  0.87421538,  0.81090356],
       [ 0.85216546,  0.85201827,  0.82712856],
       [ 0.78669288,  0.78627581,  0.76326608],
       [ 0.79210207,  0.79288072,  0.81821463],
       [ 0.9024305 ,  0.90235752,  0.84843801],
       [ 0.82825217,  0.82850497,  0.85464635],
       [ 0.87876132,  0.87861839,  0.87470816],
       [ 0.87214629,  0.87238737,  0.83172366],
       [ 0.70247813,  0.70238113,  0.77301631],
       [ 0.90982315,  0.90950979,  0.88639891],
       [ 0.85466483,  0.85501604,  0.81767274],
       [ 0.87811593,  0.87862594,  0.87703194],
       [ 0.78018934,  0.77911594,  0.80858132],
       [ 0.88986585,  0.89006211,  0.82496898]])